In [1]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = 'gpt-4.1-nano',temperature = 0)
prompt = ChatPromptTemplate(
    [
        ('system','넌 유용한 챗봇이야'),
        MessagesPlaceholder(variable_name = 'chat_history') # 기존 채팅 내역을 다 주입
        ('human','{input}')
    ]
)

memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

memory.load_memory_variables({})

ImportError: cannot import name 'RunnableLambda' from 'langchain.runnables' (c:\Users\mingyu\anaconda3\Lib\site-packages\langchain\runnables\__init__.py)

In [2]:
%pip install langchain_core

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import time
from operator import itemgetter
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import  PydanticOutputParser, StrOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

llm = ChatOpenAI(model = 'gpt-4.1-nano',temperature = 0)
prompt = ChatPromptTemplate(
    [
        ('system','넌 좀 틱틱대는 챗봇이야'),
        MessagesPlaceholder(variable_name = 'chat_history'),
        ('human','{input}'),
    ]
)
# Memory
memory = ConversationBufferMemory(return_messages= True, memory_key = 'chat_history')
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables)|
    itemgetter('chat_history')
)
chain = runnable| prompt| llm | StrOutputParser()
#메세지 스트리밍 하는 함수
def stream_msg(chain, msg):
    full_msg = ''
    print('AI: ',end = '')
    for token in chain.stream({'input':msg}):
        full_msg += token
        print(token, end = '', flush = True)
    print('\n---')
    return full_msg
input_msg = ''

In [ ]:
from operator import itemgetter
runnable = RunnablePassthrough.assign(
    chat_history = RunnableLambda(memory.load_memory_variables)|
    itemgettter('chat_history')
)